<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/99-rnn-lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lotto

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from io import BytesIO
import requests

In [ ]:
noEnd ='965' # 마지막 로또 회차 번호
url = 'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd={}'.format(noEnd)

# user-agent
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

In [ ]:
result = requests.get(url, headers={"user-agent":user_agent})

In [ ]:
result

In [ ]:
result.content

In [ ]:
f = BytesIO(result.content)
# df = pd.read_excel(f,sheet_name='excel',skiprows=3)
html_result = pd.read_html(f)

In [ ]:
html_result

In [ ]:
df = df.sort_values('seq', ascending=True)
ncount = np.zeros((1,45), dtype=float)

In [ ]:
for i in range(df.shape[0]):
  for j in range(6):
    col = df.iloc[i][str(j+1)]
    ncount[0][col-1] += 1.0

  if i == 0:
    X = ncount
  else:
    X = np.append(X, ncount, axis=0)

print('x.shape:', X.shape)

In [ ]:
df_y = df[['1','2','3','4','5','6']]
Y = df_y.values
print('x.shape:', X.shape)
print('y.shape:', Y.shape)

In [ ]:
y_onehot = []
tmponehot = np.zeros(45).tolist()
tmponehot[3] = 1.0
y_onehot.append(tmponehot)
y_onehot.append(tmponehot)

np.array(y_onehot).shape

In [ ]:
def make_onehot(value):
  onehot = np.zeros(45).tolist()
  onehot[value-1] = 1.0
  return onehot

In [ ]:
y_onehot = []
for i in range(Y.shape[0]):
  for j in range(6):
    y_onehot.append(make_onehot(Y[i][j]))

y_onehot = np.array(y_onehot)
y_onehot = np.reshape(y_onehot, (964,6,45))
print('y_onehot.shape:', y_onehot.shape)

In [ ]:
X.shape

In [ ]:
def build_dataset(x_time_series, y_time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(y_time_series) - seq_length):
        x = x_time_series[i:i + seq_length, :]
        y = y_time_series[i + seq_length - 1, :]
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)

In [ ]:
x_data, y_data = build_dataset(X, y_onehot, 6)

In [ ]:
print('x_data.shape:', x_data.shape)
print('y_data.shape:', y_data.shape)

In [ ]:
model = keras.Sequential();
model.add(keras.layers.LSTM(units=45, input_shape=(6, 45), return_sequences=True))
model.add(keras.layers.LSTM(units=45, return_sequences=True))
model.add(keras.layers.Dense(units=45, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_data, y_data, epochs=1200, verbose=0)

In [ ]:
x_test = x_data[-1,:]
x_test = np.delete(x_test, 0, axis=0)
x_test = np.append(x_test, X[-1:], axis=0)
x_test = x_test.reshape((1,6,45))
print('x_test.shape:', x_test.shape)

In [ ]:
pred_y = model.predict(x_test)

In [ ]:
for i in range(6):
  print(np.argmax(pred_y[0][i]) + 1)